# Contexto

 Atualmente, o **Instagram** é uma das principais plataformas de mídia social utilizadas para **marketing**. Optamos por utilizar **hashtags (#)** para pegar os **textos e comentários das publicações (posts)**, pois elas são uma excelente maneira de aumentar a visibilidade dos **posts** e alcançar um público-alvo específico. No nosso caso, estamos focando nas pessoas que consomem **barras de cereais**.

 Ao utilizar **hashtags relevantes nos posts**, permite-se que as pessoas interessadas nesse tema encontrem **facilmente o conteúdo**, pois as **hashtags** direcionam o conteúdo para aqueles que estão procurando **informações específicas** sobre **barras de cereais**.

 Portanto, ao adotar essa estratégia de pesquisar pelas **hashtags relevantes**, alcança-se diretamente o **público** que busca informações sobre **barras de cereais**.

 Para atingirmos um **público maior**, realizamos as pesquisas com **sinônimos de barras de cereais**, onde utilizamos **diversas hashtags relevantes**, como:

- #BarraProteica
- #BarraDeProteina
- #BarraDeCereal
- #BarraEnergética
- #BarraDeGranola
- #BarraNutritiva
- #BarradeCereais
- #BarrinhaSaudavel
- #BoldSnacks
- #WheyBar
- #BarrinhaFit

## Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm
import pandas as pd
import csv

## Lista de palavras/ termos usados para fazer a raspagem no Instagram

In [1]:
list_words = ["#barra proteica","#barra de proteina", "#barra de cereal", "#barra energética",
              "#barra de granola", "#barra nutritiva", "#barra de cereais", "#barrinha saudavel",
              "#bold snacks", "#whey bar", "barrinha fit"]
list_words = [string.replace(" ", "") for string in list_words]

## Abre o navegador e acessa o Instagram

In [3]:
driver = webdriver.Safari()
driver.get(f"https://www.instagram.com/")
time.sleep(2)

## Funções que servem para esperar até que um elemento esteja presente ou seja localizado no Instagram

- Usamos a classe **WebDriverWait** fornecida pela biblioteca do Selenium
- Ambas as funções tem um tempo de espera de **5 segundos**, e caso o elemento não for encontrado dentro de 5 segundos, uma exceção **TimeoutException**  será retornada

In [4]:
def wait_for_element(element, address):
    return WebDriverWait(driver, 5).until(EC.presence_of_element_located((element, address)))

def wait_for_elementS(element, address):
    return WebDriverWait(driver, 5).until(lambda driver: driver.find_elements(element, address))

## Preenchimento do Login (Usuário e Senha)
- O **delay** usado no campo de senha, é utilizado para adicionar um **atraso** de **1 segundo** entre o envio de cada caractere da **senha**, pois caso não haja esse delay, o campo da senha será preechido no campo do usuário

In [5]:
user_name = wait_for_element(By.CSS_SELECTOR, "input[name='username']")
user_name.send_keys("seu_user")

password = wait_for_element(By.CSS_SELECTOR, "input[name='password']")
pswd = "sua_senha"
delay = 1
for char in pswd:
    password.send_keys(char)
    time.sleep(delay)

## Clica no botão de ENTRAR , após o preenchimento do Login

In [6]:
time.sleep(5)
login_button = wait_for_element(By.CSS_SELECTOR, "button[type='submit']")
login_button.click()

## Clica no botão de AGORA NÃO, em "Salvar suas informações de login?"
- Esse botão nem sempre é exibido, então usamos o try-except: 
  - Try: Clicará no botão **Agora não** quando a mensagem **Salvar suas informações de login?** for exibida
  - Except: Caso o elemento não seja encontado, será retornado a exceção **NoSuchElementException**, onde será executado o comando **pass** (O comando pass é utilizado para indicar que nenhuma ação adicional deve ser realizada, dessa maneira, passando para a próxima célula)
                                                          

In [7]:
try:
    not_now_button = wait_for_element(By.CLASS_NAME, "_ac8f")
    not_now_button.click()
except NoSuchElementException:
    pass

## Clica no botão de AGORA NÃO, em "Ativar notificações?"
- O **not_now_buttons** retorna uma lista contendo todos os elementos que correspondem a classe **_a9--**
- O **for** irá percorrer cada elmento da lista, e quando encontrar o texto do botão **Agora não**, irá clicar no botão e interromper a interação (**break**)

In [8]:
not_now_buttons = wait_for_elementS(By.CLASS_NAME, "_a9--")
for button in not_now_buttons:
    if button.text == "Agora não":
        button.click()
        break

## Função get_search (Obtém pesquisa)
- Primeiro a funcão irá clicar na **lupa de pesquisa**
- Após clicar na lupa de pesquisa, a função irá **digitar a palavra/termo** que se deseja **pesquisar**
- Após digitar a palavra/termo que se deseja pesquisar, a função dará um **duplo clique (ENTER)** para entrar na página da **hashtag (#)** pesquisada
- Após entrar na página da hashtag (#) pesquisada, a função irá clicar no **primeiro post** encontrado

In [9]:
def get_search(list_words):
    time.sleep(3)
    button_search = wait_for_element(By.XPATH, "//*[@aria-label='Pesquisa']")
    button_search.click()
    
    time.sleep(3)
    button_search_name = wait_for_element(By.XPATH, "//*[@aria-label='Entrada da pesquisa']")
    button_search_name.send_keys(list_words)
    
    time.sleep(3)
    button_search_name.send_keys(Keys.RETURN)
    button_search_name.send_keys(Keys.RETURN)
    
    time.sleep(3)
    fisrt_post = wait_for_element(By.CLASS_NAME, "_aagu")
    fisrt_post.click()

## Função extract_text_and_comments_from_posts (Extrair texto e comentários de postagens)
- **with tqdm(total=number_of_posts, desc="Carregando posts") as pbar:** cria uma barra de progresso que irá acompanhar o loop que extrai o texto e os comentários dos posts
- O **for** criará um loop com a quantidade de posts (**number_of_posts**) que se deseja **extrair**, e dentro dele executará os seguintes passos da função:
    - Primeiro a função irá tentar (try) **extrair o texto dos post e adicionar na lista (list_text)** 
    - Depois a função irá tentar (try) **extrair os comentário de cada post e adicioar na lista (list_comments)**  
    - Depois de tentar (try) extrair o texto e comentários do post, a função irá clicar no botão **Avançar**, ou seja, irá para o próximo post
    - Nos 3 passos anteriores, caso o *try* não funcione, ou seja, caso o **elemento não seja encontrado na página web**, será executado o **exept**, onde retornará as exceções **NoSuchElementException ou TimeoutException**, onde será executado o comando **pass** (O comando pass é utilizado para indicar que nenhuma ação adicional deve ser realizada, dessa maneira, continuando o loop)
- O **if** irá adicionar um tempo de **espera** de **10 segundos a cada 50 posts**, pois caso essa condição não seja adicionada, o instagram **banirá a conta por atividade suspeita**
- Após o **for** ser executado, e ter **populado** as listas **list_text** e **list_comments**, elas serão adicionadas em um dataframe e **retornadas** para que possamos usálas fora da função

In [10]:
def extract_text_and_comments_from_posts(number_of_posts):
    list_text = []
    list_comments = []
    
    with tqdm(total=number_of_posts, desc="Carregando posts") as pbar:
        for x in range(number_of_posts):
            time.sleep(0.5)
            try:
                extract_text = wait_for_element(By.CLASS_NAME, "_a9zs")  
                list_text.append(extract_text.text)
            except (NoSuchElementException, TimeoutException):
                pass 

            try:
                comments = driver.find_elements(By.XPATH, "//span[contains(@class, '_aacl')]")
                comment_texts = [comment.text for comment in comments]
                list_comments.extend(comment_texts)
            except (NoSuchElementException, TimeoutException):
                pass

            try:
                button_search = wait_for_element(By.XPATH, "//*[@aria-label='Avançar']")
                button_search.click()
            except (NoSuchElementException, TimeoutException):
                pass

            pbar.update(1)

            if (x + 1) % 50 == 0:
                time.sleep(10)

    df_text = pd.DataFrame(list_text, columns=["Text"])
    df_comments = pd.DataFrame(list_comments, columns=["Comentários"])
    return df_text, df_comments


## Função execute_functions (Executa as funções)
- Essa função irá executar as funções **get_search** e **extract_text_and_comments_from_posts** 
- A função também transformará os dataframes em listas, para que possamos adicionálas nas listas **all_text** e **all_comments** e assim usarmos elas fora da função

In [11]:
def execute_functions(word,number_of_posts):
    all_text = []
    all_comments = []
    
    get_search(word)
    list_text_posts, list_comments_posts = extract_text_and_comments_from_posts(number_of_posts)
    all_text.extend(list_text_posts.values.tolist())
    all_comments.extend(list_comments_posts.values.tolist())
    return all_text, all_comments

## Função save_csv (Salva em CSV)
- A função **salva os arquivos em CSVs** na máquina da pessoa (caso a pasta esteja no mesmo local que o Jupyter)
- E **trasnforma** as listas em **DataFrames**

In [12]:
def save_csv(name, df):
    with open(file_name, 'w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(pd.DataFrame(df).columns)
        writer.writerows(pd.DataFrame(df).values)
    return f"Salvo: {file_name}"

## Aqui é chamado a função execute_functions e save_csv
- Na função **execute_functions** é **necessário** passar **2 parâmetros**, sendo:
  - 1º a palavra/ termo que se deseja buscar (no casso nossas palavras se encontram na lista **list_words**, onde será **necessário alterar** apenas o **index** da lista **list_words[x]**, onde **x é o index**)
  - 2º a **quantidade de posts** que se deseja buscar 
- Na função **save_csv** é **necessáio** passar **2 parâmetros**, sendo:
  - 1º o **nome do arquivo** como se deseja **salvar** (nesse caso salvamos todos os arquivos como **TEXTn.csv** e **COMMENTSn.csv**, apenas mudando o número (**n**))
  - 2º a lista que se deseja salvar (as mesmas que são **definas** para quando se chamar a função **execute_functions**, em todos os **save_csv** é usado **df_TEXT** e **df_COMMENTS**)
  
###### Obs: É necessário que execute apenas 1 único execute_functions por vez na mesma página web, pois se executar mais de um, o Instagram banirá a conta por atividade suspeita 

In [13]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[0], 500)

csv_text = save_csv("TEXT0.csv", df_TEXT)
csv_comments = save_csv("COMMENTS0.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[0], 500)\n\ncsv_text = save_csv("TEXT0.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS0.csv", df_COMMENTS)'

In [14]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[1], 500)

csv_text = save_csv("TEXT1.csv", df_TEXT)
csv_comments = save_csv("COMMENTS1.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[1], 500)\n\ncsv_text = save_csv("TEXT1.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS1.csv", df_COMMENTS)'

In [15]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[2], 500)

csv_text = save_csv("TEXT2.csv", df_TEXT)
csv_comments = save_csv("COMMENTS2.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[2], 500)\n\ncsv_text = save_csv("TEXT2.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS2.csv", df_COMMENTS)'

In [16]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[3], 500)

csv_text = save_csv("TEXT3.csv", df_TEXT)
csv_comments = save_csv("COMMENTS3.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[3], 500)\n\ncsv_text = save_csv("TEXT3.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS3.csv", df_COMMENTS)'

In [17]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[4], 200)

csv_text = save_csv("TEXT4.csv", df_TEXT)
csv_comments = save_csv("COMMENTS4.csv", df_COMMENT)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[4], 200)\n\ncsv_text = save_csv("TEXT4.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS4.csv", df_COMMENT)'

In [18]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[5], 500)

csv_text = save_csv("TEXT5.csv", df_TEXT)
csv_comments = save_csv("COMMENTS5.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[5], 500)\n\ncsv_text = save_csv("TEXT5.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS5.csv", df_COMMENTS)'

In [19]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[6], 1000)

csv_text = save_csv("TEXT6.csv", df_TEXT)
csv_comments = save_csv("COMMENTS6.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[6], 1000)\n\ncsv_text = save_csv("TEXT6.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS6.csv", df_COMMENTS)'

In [20]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[7], 700)

csv_text = save_csv("TEXT7.csv", df_TEXT)
csv_comments = save_csv("COMMENTS7.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[7], 700)\n\ncsv_text = save_csv("TEXT7.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS7.csv", df_COMMENTS)'

In [21]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[8], 150)

csv_text = save_csv("TEXT8.csv", df_TEXT)
csv_comments = save_csv("COMMENTS8.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[8], 150)\n\ncsv_text = save_csv("TEXT8.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS8.csv", df_COMMENTS)'

In [22]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[9], 400)

csv_text = save_csv("TEXT9.csv", df_TEXT)
csv_comments = save_csv("COMMENTS9.csv", df_COMMENTS)"""

'df_TEXT, df_COMMENTS = execute_functions(list_words[9], 400)\n\ncsv_text = save_csv("TEXT9.csv", df_TEXT)\ncsv_comments = save_csv("COMMENTS9.csv", df_COMMENTS)'

In [23]:
"""df_TEXT, df_COMMENTS = execute_functions(list_words[10], 600)

csv_text = save_csv("TEXT10.csv", df_TEXT)
csv_comments = save_csv("COMMENTS10.csv", df_COMMENT)"""

Carregando posts: 100%|██████████| 600/600 [07:57<00:00,  1.26it/s]


## Concatenar todos os arquivos em um único arquivo
- O primeiro **for** criára um loop onde irá **ler** todos os arquivos **CSVs** que **começam** com **TEXT**, e adicionar na lista **list_df_TEXT**
- O segundo **for** criára um loop onde irá **ler** todos os arquivos **CSVs** que **começam** com **COMMENTS**, e adicionar na lista **list_df_COMMENTS**
- Após isso, o **df_concatenated_TEXT** e **df_concatenated_COMMENTS** irão **concatenar** todas as listas salvas dentro de **list_df_TEXT** e **list_df_COMMENTS**
- Após isso, foi chamado a função **save_csv** para salavar os **DataFrames concatenados** 
- Por fim, foi chamado os DataFrames concatenados abaixo, para analisarmos como ficaram

In [36]:
list_df_TEXT = []
for file in range(11):
    list_df_TEXT.append(pd.read_csv(f"TEXT{file}.csv", sep=',', encoding='utf-8'))
    
list_df_COMMENTS = []
for file in range(11):
    list_df_COMMENTS.append(pd.read_csv(f"COMMENTS{file}.csv", sep=',', encoding='utf-8'))

In [46]:
df_concatenated_TEXT = pd.concat(list_df_text).rename(columns={'0': 'TEXT'})
df_concatenated_COMMENTS = pd.concat(list_df_COMMENTS).rename(columns={'0': 'COMMENTS'})

In [47]:
csv_TEXT = save_csv("df_concatenated_TEXT.csv", df_concatenated_TEXT)
csv_COMMENTS = save_csv("df_concatenated_COMMENTS.csv", df_concatenated_COMMENTS)

In [49]:
df_concatenated_COMMENTS

,COMMENTS
0,Amo
1,🏆🏆🏆
2,👏😍
3,😍
4,Bom dia.😘❤️
...,...
1353,"Muito boa Vi 😋😋😋, já tinha tentado fazer uma a..."
1354,Olha que delícia 😍😍😋
1355,Nossa obggg @negahealthy_ 😍😍
1356,@mari_krunner aqui gatona 😘😘😘


In [66]:
df_concatenated_COMMENTS

,COMMENTS
0,Amo
1,🏆🏆🏆
2,👏😍
3,😍
4,Bom dia.😘❤️
...,...
1353,"Muito boa Vi 😋😋😋, já tinha tentado fazer uma a..."
1354,Olha que delícia 😍😍😋
1355,Nossa obggg @negahealthy_ 😍😍
1356,@mari_krunner aqui gatona 😘😘😘
